In [1]:
from process_bigraph import ProcessTypes

CORE = ProcessTypes()

In [2]:
import warnings
from basico import * 
from process_bigraph import Process


warnings.filterwarnings("ignore", category=FutureWarning)


class ODECopasi(Process):
    config_schema = {
        'model_file': 'string'
    }

    def __init__(self, config=None, core=CORE):
        super().__init__(config, core)
        self.model = load_model(self.config['model_file'])
        self.reaction_names = get_reactions(model=self.model).index.tolist()
        self.species_names = get_species(model=self.model).index.tolist()

    def initial_state(self):
        initial_concentrations = {
            species_name: get_species(species_name, model=self.model).initial_concentration[0]
            for species_name in self.species_names
        }

        initial_derivatives = {
            rxn_id: get_reactions(rxn_id, model=self.model).flux[0]
            for rxn_id in self.reaction_names
        }

        return {
            'species_concentrations': initial_concentrations,
            'reaction_fluxes': initial_derivatives,
            'time': 0.0
        }

    def inputs(self):
        concentrations_type = {
            name: 'float' for name in self.species_names
        }
        return {
            'species_concentrations': concentrations_type,
            'time': 'float'
        }

    def outputs(self):
        concentrations_type = {
            name: 'float' for name in self.species_names
        }

        reaction_fluxes_type = {
            reaction_name: 'float' for reaction_name in self.reaction_names
        }

        return {
            'species_concentrations': concentrations_type,
            'reaction_fluxes': reaction_fluxes_type,
            'time': 'float'
        }

    def update(self, inputs, interval):
        for cat_id, value in inputs['species_concentrations'].items():
            set_type = 'concentration'
            species_config = {
                'name': cat_id,
                'model': self.model,
                set_type: value
            }
            set_species(**species_config)

        # run model for "interval" length; we only want the state at the end
        tc = run_time_course(
            start_time=inputs['time'],
            duration=interval,
            update_model=True,
            model=self.model
        )

        results = {'time': interval}
        results['species_concentrations'] = {
            mol_id: float(get_species(
                name=mol_id,
                exact=True,
                model=self.model
            ).concentration[0])
            for mol_id in self.species_names
        }

        results['reaction_fluxes'] = {
            rxn_id: float(get_reactions(
                name=rxn_id,
                model=self.model
            ).flux[0])
            for rxn_id in self.reaction_names
        }

        return results


CORE.process_registry.register('ode-copasi', ODECopasi)

In [3]:
import logging
import os
from pathlib import Path

import cobra
from cobra.io import read_sbml_model
from process_bigraph import Process, Composite
import numpy as np


logging.getLogger('cobra').setLevel(logging.ERROR)


class FBA(Process):
    config_schema = {
        'model_file': 'string',
        'objective': {
            'domain': 'string',  # either protein or mrna
            'name': 'string',  # specific to the model: i.e., LacI
            'scaling_factor': 'float'
        }
    }

    def __init__(self, config=None, core=CORE):
        super().__init__(config, core)

        # create model
        model_file = self.config['model_file']
        data_dir = Path(os.path.dirname(model_file))
        path = data_dir / model_file.split('/')[-1]
        self.model = read_sbml_model(str(path.resolve()))

        # parse objective
        self.objective_domain = self.config['objective']['domain']
        self.objective_name = self.config['objective']['name']
        self.scaling_factor = self.config['objective'].get('scaling_factor', 10)

        # set objectives
        self.model.objective = {
            self.model.reactions.get_by_id(reaction.id): np.random.random()  # TODO: make this more realistic
            for reaction in self.model.reactions
        }

        # set even bounds
        for reaction in self.model.reactions:
            rand_bound = np.random.random()
            self.model.reactions.get_by_id(reaction.id).lower_bound = -rand_bound  # TODO: What to do here?
            self.model.reactions.get_by_id(reaction.id).upper_bound = rand_bound

    def initial_state(self):
        initial_fluxes = {}
        initial_solution = self.model.optimize()
        if initial_solution.status == 'optimal':
            initial_fluxes = {
                reaction.name: reaction.flux
                for reaction in self.model.reactions
            }

        return {'fluxes': initial_fluxes}

    def inputs(self):
        return {'reaction_fluxes': 'tree[float]'}

    def outputs(self):
        return {'fluxes': 'tree[float]'}

    def update(self, state, interval):
        for reaction_name, reaction_flux in state['reaction_fluxes'].items():
            for reaction in self.model.reactions:
                if reaction.name == reaction_name:
                    # 1. reset objective weights according to reaction fluxes directly
                    self.model.objective = {
                        self.model.reactions.get_by_id(reaction.id): reaction_flux
                    }

                    # 2. set lower bound with scaling factor and reaction fluxes
                    self.model.reactions.get_by_id(reaction.id).lower_bound = -self.scaling_factor * abs(reaction_flux)  # / (5 + abs(reaction_flux))

        # 3. solve for fluxes
        output_state = {}
        solution = self.model.optimize()
        if solution.status == "optimal":
            data = dict(zip(
                list(state['reaction_fluxes'].keys()),
                list(solution.fluxes.to_dict().values())
            ))
            output_state['fluxes'] = data

            # TODO: do we want to instead scale by input flux?
            # for reaction in self.model.reactions:
            #     flux = solution.fluxes[reaction.id]
            #     for reaction_name, reaction_flux in state['reaction_fluxes'].items():
            #         if reaction.name == reaction_name:
            #             fluxes[reaction.name] = flux * reaction_flux

        return output_state


CORE.process_registry.register('fba', FBA)

ImportError: cannot import name 'object' from 'numpy' (/Users/alexanderpatrie/Library/Caches/pypoetry/virtualenvs/bio-bundles-WKcrZPME-py3.10/lib/python3.10/site-packages/numpy/__init__.py)

In [4]:
fp = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml'
fp2 = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Beard2005_Mitochondrial_Respiration.xml'
fp3 = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Mitchell2013/BIOMD0000000498_url.xml'


doc = {
        'ode': {
            '_type': 'process',
            'address': f'local:ode-copasi',
            'config': {
                'model_file': fp3
            },
            'inputs': {
                'time': ['time_store'],
                'species_concentrations': ['species_concentrations_store']
            },
            'outputs': {
                'time': ['time_store'],
                'species_concentrations': ['species_concentrations_store'],
                'reaction_fluxes': ['reaction_fluxes_store']
            }
        },
        'fba': {
            '_type': 'process',
            'address': f'local:fba',
            'config': {
                'model_file': fp3,
            },
            'inputs': {
                'reaction_fluxes': ['reaction_fluxes_store']
            },
            'outputs': {
                'fluxes': ['fluxes_store']
            }
        },
        'emitter': {
            '_type': 'step',
            'address': 'local:ram-emitter',
            'config': {
                'emit': {
                    'time': 'float',
                    'species_concentrations': 'tree[float]',
                    'reaction_fluxes': 'tree[float]',
                    'fluxes': 'tree[float]'
                }
            },
            'inputs': {
                'time': ['time_store'],
                'species_concentrations': ['species_concentrations_store'],
                'reaction_fluxes': ['reaction_fluxes_store'],
                'fluxes': ['fluxes_store']
            }
        }
    }

In [ ]:
composition = Composite(config={'state': doc}, core=CORE)

# save initial state 
composition.save(filename='dfba_initial.json', outdir='.')

In [ ]:
composition.run(200)

# save final state
composition.save(filename='dfba_final.json', outdir='.')

# get result from emitter
result = composition.gather_results()[('emitter',)]

In [ ]:
result[:2]

In [ ]:
import json 

with open('dfba_final.json', 'r') as f:
    checkpoint = json.load(f)
    
composition2 = Composite(config={'state': checkpoint}, core=CORE)

In [ ]:
composition2.state

In [1]:
!rm ../tests/fixtures/smoldyn/molecule_out.txt

rm: ../tests/fixtures/smoldyn/molecule_out.txt: No such file or directory


In [2]:
!rm ../tests/fixtures/smoldyn/lip2_out.txt && rm ../tests/fixtures/smoldyn/lip2.simularium

rm: ../tests/fixtures/smoldyn/lip2_out.txt: No such file or directory


In [3]:
!rm ../tests/fixtures/smoldyn/lip_out.txt && rm ../tests/fixtures/smoldyn/lip.simularium

rm: ../tests/fixtures/smoldyn/lip.simularium: No such file or directory


In [4]:
import smoldyn

fp = '/Users/alexanderpatrie/Desktop/repos/bio-bundles/tests/fixtures/smoldyn/lip2.txt'      # lip.txt'
fp2 = '/Users/alexanderpatrie/Desktop/repos/bio-bundles/tests/fixtures/smoldyn/tracking.txt'
sim = smoldyn.Simulation.fromFile(fp)

--------------------------------------------------------------
Running Smoldyn 2.73

CONFIGURATION FILE
 Path: '/Users/alexanderpatrie/Desktop/repos/bio-bundles/tests/fixtures/smoldyn/'
 Name: 'lip2.txt'
 Reading file: '/Users/alexanderpatrie/Desktop/repos/bio-bundles/tests/fixtures/smoldyn/lip2.txt'
Error reading file in line 81
line:     action both reflect 
file: /Users/alexanderpatrie/Desktop/repos/bio-bundles/tests/fixtures/smoldyn/lip2.txt
Message: action format: species[(state)] face action


Simulation:: Fatal error: failed to initialize Simulation.


In [4]:
sim.runSim()

Libsmoldyn notification from smolRunSim: Simulation complete


<ErrorCode.ok: 0>

In [5]:
!ls ../tests/fixtures/smoldyn

MinE.txt                  molecule.simularium       tracking_listmols.txt
lip.txt                   molecule.txt              tracking_listmols_out.txt
lip_counts_out.txt        tracking.simularium       tracking_out.txt
lip_out.txt               tracking.txt              tracking_trackmol.txt


In [6]:
from IPython.display import Image
import numpy as np
from simulariumio.smoldyn import SmoldynConverter, SmoldynData
from simulariumio import (
    UnitData, 
    MetaData, 
    DisplayData, 
    DISPLAY_TYPE, 
    ModelMetaData, 
    BinaryWriter, 
    InputFileData,
)
from simulariumio.filters import TranslateFilter


box_size = 1000.

example_data = SmoldynData(
    meta_data=MetaData(
        box_size=np.array([box_size, box_size, box_size]),
        trajectory_title="Some parameter set",
        model_meta_data=ModelMetaData(
            title="LIP",
            version="8.1",
            authors="Alexander A. Patrie",
            description=(
                "Based on BIOMD0000000498 - Mitchell, Mendes PLoS Comput. Biol. 2013 Nov; 9(11): e1003299"
            ),
            doi="10.1016/j.bpj.2016.02.002",
            source_code_url="https://github.com/simularium/simulariumio",
            source_code_license_url="https://github.com/simularium/simulariumio/blob/main/LICENSE",
            input_data_url="https://allencell.org/path/to/native/engine/input/files",
            raw_output_data_url="https://allencell.org/path/to/native/engine/output/files",
        ),
    ),
    smoldyn_file=InputFileData(
        file_path="/Users/alexanderpatrie/Desktop/repos/bio-bundles/tests/fixtures/smoldyn/lip_out.txt"
    ),
    # display_data={
    #     "Fe2_ION(solution)": DisplayData(
    #         name="Fe2+",
    #         radius=0.3,  # 0.0001 for µ
    #         display_type=DISPLAY_TYPE.SPHERE,
    #         color="#00FF00"
    #     ),
    #     "Ferritin(solution)": DisplayData(
    #         name="Ferritin",
    #         radius=0.8,  # 0.006 for µ
    #         display_type=DISPLAY_TYPE.SPHERE,
    #         color="#FF0000"
    #     ),
    #     "Ferritin_Fe2_ION(solution)": DisplayData(
    #         name="Ferritin_Fe2+",
    #         radius=0.8,
    #         display_type=DISPLAY_TYPE.SPHERE,
    #         color="#800080"
    #     ),
    # },
    time_units=UnitData("ns"),  # nanoseconds
    spatial_units=UnitData("nm"),  # nanometers
    center=True,
)

In [7]:
# from simulariumio import ScatterPlotData
# 
# example_scatter_plot = ScatterPlotData(
#     title="Test Scatterplot 1",
#     xaxis_title="time (ns)",
#     yaxis_title="concentration (uM)",
#     xtrace=np.array(list(range(10))),
#     ytraces={
#         "agent1": 100 * np.random.uniform(size=(10)),
#         "agent2": 100 * np.random.uniform(size=(10)),
#         "agent3": 100 * np.random.uniform(size=(10)),
#     },
#     render_mode="lines"
# )

In [8]:
converter = SmoldynConverter(example_data)
# converter.add_plot(example_scatter_plot)
converter.add_number_of_agents_plot()
converter.save("../tests/fixtures/smoldyn/lip", binary=False)

Reading Smoldyn Data -------------
Reading Scatter Plot Data -------------
Converting Trajectory Data to JSON -------------
Writing JSON -------------
saved to ../tests/fixtures/smoldyn/lip.simularium


In [ ]:
import json 

with open('../tests/fixtures/smoldyn/lip.simularium', 'r') as f:
    trajectory = json.load(f)
    
trajectory.keys()

In [ ]:
for i, d in enumerate(trajectory.get('spatialData').get('bundleData')):
    print(i, d['time'])

In [14]:
import requests

run_id = '61fea4a08c1e3dc95a79802e'
name = 'simulation.sedml/report'
headers = {'accept': 'application/json'}
params = {
    'run_id': run_id,
    'dataset_name': name,
}
url = f'https://simdata.api.biosimulations.org/datasets/{run_id}/data'

resp = requests.get(url, params={'dataset_name': name}, headers=headers)

In [23]:
data = resp.json().get('values')

In [31]:
t = data[:601]

In [32]:
len(data) / 601

7.0

In [33]:
601*2

1202

In [35]:
from mass.io import sbml

ImportError: cannot import name 'float_' from 'numpy' (/Users/alexanderpatrie/Library/Caches/pypoetry/virtualenvs/bio-bundles-WKcrZPME-py3.10/lib/python3.10/site-packages/numpy/__init__.py)

In [5]:
from basico import * 

In [6]:
m = load_model(fp3)

In [10]:
get_species(model=m).index.tolist()

['Hamp',
 'Fe-FT',
 'FT',
 'FT1',
 'HO-1',
 'Heme',
 'LIP',
 'Fpn',
 'IRP',
 'Tf-Fe_intercell',
 'TfR',
 'Tf-Fe-TfR1',
 'HFE',
 'HFE-TfR',
 'Tf-Fe-TfR2',
 '2(Tf-Fe)-TfR1',
 '2HFE-TfR',
 '2HFE-TfR2',
 '2(Tf-Fe)-TfR2',
 'TfR2',
 'Heme_intercell']